# Biostat 257 Homework 5

**Due June 8 @ 11:59PM**

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake)


Again we continue with the linear mixed effects model (LMM)
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma}_i + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effects predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effects predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma}_i \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

The log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ is 
$$
    \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma_0^2) = - \frac{n_i}{2} \log (2\pi) - \frac{1}{2} \log \det \boldsymbol{\Omega}_i - \frac{1}{2} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta}),
$$
where
$$
    \boldsymbol{\Omega}_i = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T.
$$
Given $m$ independent data points $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$, $i=1,\ldots,m$, we seek the maximum likelihood estimate (MLE) by maximizing the log-likelihood
$$
\ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma_0^2) = \sum_{i=1}^m \ell_i(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma_0^2).
$$

In HW4, we used the nonlinear programming (NLP) approach (Newton type algorithms) for optimization. In this assignment, we derive and implement an expectation-maximization (EM) algorithm for the same problem.

In [2]:
# load necessary packages; make sure install them first
using BenchmarkTools, Distributions, LinearAlgebra, Random, Revise

## Q1. (10 pts) Refresher on normal-normal model

Assume the conditional distribution
$$
\mathbf{y} \mid \boldsymbol{\gamma} \sim N(\mathbf{X} \boldsymbol{\beta} + \mathbf{Z} \boldsymbol{\gamma}, \sigma^2 \mathbf{I}_n)
$$
and the prior distribution
$$
\boldsymbol{\gamma} \sim N(\mathbf{0}_q, \boldsymbol{\Sigma}).
$$
By the Bayes theorem, the posterior distribution is
\begin{eqnarray*}
f(\boldsymbol{\gamma} \mid \mathbf{y}) &=& \frac{f(\mathbf{y} \mid \boldsymbol{\gamma}) \times f(\boldsymbol{\gamma})}{f(\mathbf{y})}, \end{eqnarray*}
where $f$ denotes corresponding density. 

Show that the posterior distribution of random effects $\boldsymbol{\gamma}$ is a multivariate normal with mean
\begin{eqnarray*}
\mathbb{E} (\boldsymbol{\gamma} \mid \mathbf{y}) &=& \sigma^{-2} (\sigma^{-2} \mathbf{Z}^T \mathbf{Z} + \boldsymbol{\Sigma}^{-1})^{-1 } \mathbf{Z}^T (\mathbf{y} - \mathbf{X} \boldsymbol{\beta}) \\
&=& \boldsymbol{\Sigma} \mathbf{Z}^T (\mathbf{Z} \boldsymbol{\Sigma} \mathbf{Z}^T + \sigma^2 \mathbf{I})^{-1} (\mathbf{y} - \mathbf{X} \boldsymbol{\beta})
\end{eqnarray*}
and covariance
\begin{eqnarray*}
\text{Var} (\boldsymbol{\gamma} \mid \mathbf{y}) &=& (\sigma^{-2} \mathbf{Z}^T \mathbf{Z} + \boldsymbol{\Sigma}^{-1})^{-1} \\
&=& \boldsymbol{\Sigma} - \boldsymbol{\Sigma} \mathbf{Z}^T (\mathbf{Z} \boldsymbol{\Sigma} \mathbf{Z}^T + \sigma^2 \mathbf{I})^{-1} \mathbf{Z} \boldsymbol{\Sigma}.
\end{eqnarray*}

$$f( \mathbf{\gamma} | \mathbf{y}) \propto f( \mathbf{y} | \mathbf{\gamma}) f(\mathbf{\gamma})$$
$$ \propto \exp(-\frac{1}{2}\mathbf{\gamma}^T\mathbf{\Sigma}^{-1}\mathbf{\gamma} - \frac{1}{2} \sigma^{-2} \mathbf{\gamma}^T \mathbf{Z}^T\mathbf{Z}\mathbf{\gamma} - \sigma^{-2}\mathbf{\gamma}^T\mathbf{Z}^T(\mathbf{y} - \mathbf{x\beta}))$$
$$ \propto \exp(-\frac{1}{2}\mathbf{\gamma}^T(\mathbf{\Sigma}^{-1} + \sigma^{-2}\mathbf{Z}^T\mathbf{Z})\mathbf{\gamma}^T - \sigma^{-2}\mathbf{\gamma}^T\mathbf{Z}^T(\mathbf{y}- \mathbf{X\beta}))$$
Thus,
$$ \mathbf{\gamma} | \mathbf{y} \sim N((\sigma^{-2}\mathbf{Z}\mathbf{Z}^T + \mathbf{\Sigma}^{-1})^{-1}\sigma^{-2}\mathbf{Z}^T(\mathbf{y} - \mathbf{X\beta}), (\sigma^{-2}\mathbf{Z}^T\mathbf{Z} + \Sigma^{-1})^{-1}) $$
Expectation:
$$\mathbb{E}(\mathbf{\gamma} | \mathbf{y}) = (\sigma^{-2}\mathbf{Z}\mathbf{Z}^T + \mathbf{\Sigma}^{-1})^{-1}\sigma^{-2}\mathbf{Z}^T(\mathbf{y} - \mathbf{X\beta})$$
$$ = \mathbf{\Sigma Z}^T(\sigma^{-2}\mathbf{I} - \sigma^{-2}(\sigma^2\mathbf{I} + \mathbf{Z\Sigma Z}^T)^{-1}\mathbf{Z\Sigma Z}^T)(\mathbf{y} - \mathbf{X\beta})$$
$$ = \mathbf{\Sigma Z}^T(\sigma^2\mathbf{I} + \mathbf{Z \Sigma Z}^T)^{-1}(\sigma^{-2}\mathbf{I}(\sigma^2\mathbf{I} + \mathbf{Z \Sigma Z}^T) - \sigma^{-2}\mathbf{Z \Sigma Z}^T)(\mathbf{y} - \mathbf{X\beta})$$
$$ = \mathbf{\Sigma Z}^T(\sigma^2\mathbf{I} + \mathbf{Z \Sigma Z}^T)^{-1}(\mathbf{y} - \mathbf{X\beta})$$
For variance, Woodbury Identity yields:
$$ Var(\mathbf{\gamma} | \mathbf{y}) = \mathbf{\Sigma} - \mathbf{\Sigma}\mathbf{Z}^T(\sigma^2\mathbf{I} + \mathbf{Z\Sigma Z}^T)^{-1}\mathbf{Z\Sigma}$$

## Q2. (20 pts) Derive EM algorithm

1. Write down the complete log-likelihood
$$
\sum_{i=1}^m \log f(\mathbf{y}_i, \boldsymbol{\gamma}_i \mid \boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)
$$

$$ \ell(\mathbf{y_i, \gamma_i} | \mathbf{\beta, \Sigma}, \sigma^2) = \sum(\log f(\mathbf{y_i} | \mathbf{\gamma_i}, \mathbf{\beta}, \sigma^2) + \log f (\mathbf{\gamma_i} | \mathbf{\Sigma}))$$
$$ = \sum (-\frac{n_i}{2}\log(2\pi) -\frac{1}{2} \log \det (\sigma^2\mathbf{I}_{n_i}) -\frac{1}{2}(\mathbf{y_i} - \mathbf{X_i\beta} - \mathbf{Z_i \gamma _i})^T\sigma^{-2}\mathbf{I_{n_i}} (\mathbf{y_i} - \mathbf{X_i\beta} - \mathbf{Z_i \gamma _i}) - \frac{q}{2}\log(2\pi) - \frac{1}{2} \log \det \mathbf{\Sigma} - \frac{1}{2} \mathbf{\gamma}_i^T\mathbf{\Sigma}^{-1}\mathbf{\gamma}_i)$$
2. Derive the $Q$ function (E-step).
$$
Q(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2 \mid \boldsymbol{\beta}^{(t)}, \boldsymbol{\Sigma}^{(t)}, \sigma^{2(t)}).$$

$$Q(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2 \mid \boldsymbol{\beta}^{(t)}, \boldsymbol{\Sigma}^{(t)}, \sigma^{2(t)}) = \mathbb{E}_{\mathbf{\gamma} | \mathbf{y}, \boldsymbol{\beta}^{(t)}, \boldsymbol{\Sigma}^{(t)}, \sigma^{2(t)}}( \ell(\mathbf{y_i, \gamma_i} | \mathbf{\beta, \Sigma}, \sigma^2))$$
$$ = - \sum \frac{n_i}{2}\log(2\pi) - \sum \frac{1}{2} \log \det (\sigma^2\mathbf{I}_{n_i}) - \frac{1}{2}\mathbb{E}((\mathbf{y_i} - \mathbf{X_i\beta} - \mathbf{Z_i \gamma _i})^T\sigma^{-2}\mathbf{I_{n_i}} (\mathbf{y_i} - \mathbf{X_i\beta} - \mathbf{Z_i \gamma _i})) - \sum \frac{q}{2}\log(2\pi) - \sum \frac{1}{2} \log \det \mathbf{\Sigma} - \sum \mathbb{E}(\frac{1}{2} \mathbf{\gamma}_i^T\mathbf{\Sigma}^{-1}\mathbf{\gamma}_i)$$
$$ =  - \sum \frac{n_i}{2}\log(2\pi) - \sum \frac{1}{2} \log \det (\sigma^2\mathbf{I}_{n_i}) - \frac{1}{2\sigma^2}[(\mathbf{y_i} - \mathbf{X_i\beta})^T(\mathbf{y_i} - \mathbf{X_i\beta}) -2(\mathbf{y_i} - \mathbf{X_i\beta})\mathbf{Z}\mathbb{E}(\mathbf{\gamma} | \mathbf{y}) + \mathbb{E}(\mathbf{\gamma} | \mathbf{y})^T\mathbf{Z}_i^T\mathbf{Z}_i\mathbb{E}(\mathbf{\gamma} | \mathbf{y})]$$
$$- \sum \frac{q}{2}\log(2\pi) - \sum \frac{1}{2} \log \det \mathbf{\Sigma} - \sum \mathbb{E}(\frac{1}{2} \mathbf{\gamma}_i^T\mathbf{\Sigma}^{-1}\mathbf{\gamma}_i)$$
3. Derive the EM (or ECM) update of $\boldsymbol{\beta}$, $\boldsymbol{\Sigma}$, and $\sigma^2$ (M-step). 
For better formatting, let:
$$\mathbb{E}(\mathbf{r}^t\mathbf{r}) = (\mathbf{y_i} - \mathbf{X_i\beta})^T(\mathbf{y_i} - \mathbf{X_i\beta}) -2(\mathbf{y_i} - \mathbf{X_i\beta})\mathbf{Z}\mathbb{E}(\mathbf{\gamma} | \mathbf{y}) + \mathbb{E}(\mathbf{\gamma} | \mathbf{y})^T\mathbf{Z}_i^T\mathbf{Z}_i\mathbb{E}(\mathbf{\gamma} | \mathbf{y}) $$

$$\frac{\partial Q}{\partial \mathbf{\beta}} = \sum \frac{1}{\sigma^2}(\mathbf{y}_i - \mathbf{X_i\beta}^T)\mathbf{X}_i + 2\mathbf{X}_i^T\mathbf{Z_i}\mathbb{E}(\mathbf{\gamma} | \mathbf{y})) = 0$$
$$\mathbf{\beta} = \frac{\sum (\mathbf{X}_i^T\mathbf{y}_i + \mathbf{X}_i^T\mathbf{Z}_i\mathbb{E}(\mathbf{\gamma} | \mathbf{y}))}{\sum(\mathbf{X}_i^T\mathbf{X}_i)}$$

$$\frac{\partial Q}{\partial \sigma^2} = -\frac{1}{2}\sum(\frac{n_i}{\sigma^2} - \frac{1}{\sigma^4}\mathbb{E}(\mathbf{r}^T\mathbf{r})) = 0$$
$$\sigma^2 = \frac{\sum \mathbb{E}(\mathbf{r}^T\mathbf{r})}{\sum n_i}$$

$$\frac{\partial Q}{\partial \Sigma} = m - \sum \mathbf{\Sigma}^{-1}Var(\mathbf{\gamma} | \mathbf{y})\mathbf{\Sigma}^{-1} - \sum \mathbb{E}(\mathbf{\gamma} | \mathbf{y})\mathbb{E}(\mathbf{\gamma} | \mathbf{y})^T\mathbf{\Sigma}^{-1} = 0$$
$$\mathbf{\Sigma} = \frac{1}{m}(\sum Var(\mathbf{\gamma} | \mathbf{y}) + \sum \mathbb{E}(\mathbf{\gamma} | \mathbf{y})\mathbb{E}(\mathbf{\gamma} | \mathbf{y})^T)$$

## Q3. (20 pts) Objective of a single datum

We modify the code from HW4 to evaluate the objective, the conditional mean of $\boldsymbol{\gamma}$, and the conditional variance of $\boldsymbol{\gamma}$. Start-up code is provided below. You do _not_ have to use this code.

In [10]:
# define a type that holds an LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y          :: Vector{T}
    X          :: Matrix{T}
    Z          :: Matrix{T}
    # posterior mean and variance of random effects γ
    μγ         :: Vector{T} # posterior mean of random effects
    νγ         :: Matrix{T} # posterior variance of random effects
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    yty        :: T
    rtr        :: Vector{T}
    xty        :: Vector{T}
    zty        :: Vector{T}
    ztr        :: Vector{T}
    ltztr      :: Vector{T}
    xtr        :: Vector{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    ltztzl     :: Matrix{T}
    storage_qq :: Matrix{T}
end

"""
    LmmObs(y::Vector, X::Matrix, Z::Matrix)

Create an LMM datum of type `LmmObs`.
"""
function LmmObs(
    y::Vector{T}, 
    X::Matrix{T}, 
    Z::Matrix{T}) where T <: AbstractFloat
    n, p, q = size(X, 1), size(X, 2), size(Z, 2)
    μγ         = Vector{T}(undef, q)
    νγ         = Matrix{T}(undef, q, q)
    yty        = abs2(norm(y))
    rtr        = Vector{T}(undef, 1)
    xty        = transpose(X) * y
    zty        = transpose(Z) * y
    ztr        = similar(zty)
    ltztr      = similar(zty)
    xtr        = Vector{T}(undef, p)
    storage_p  = similar(xtr)
    storage_q  = Vector{T}(undef, q)
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    ltztzl     = similar(ztz)
    storage_qq = similar(ztz)
    LmmObs(y, X, Z, μγ, νγ, 
        yty, rtr, xty, zty, ztr, ltztr, xtr,
        storage_p, storage_q, 
        xtx, ztx, ztz, ltztzl, storage_qq)
end

"""
    logl!(obs::LmmObs, β, Σ, L, σ², updater = false)

Evaluate the log-likelihood of a single LMM datum at parameter values `β`, `Σ`, 
and `σ²`. The lower triangular Cholesky factor `L` of `Σ` must be supplied too.
The fields `obs.μγ` and `obs.νγ` are overwritten by the posterior mean and 
posterior variance of random effects. If `updater==true`, fields `obs.ztr`, 
`obs.xtr`, and `obs.rtr` are updated according to input parameter values. 
Otherwise, it assumes these three fields are pre-computed. 
"""
function logl!(
        obs     :: LmmObs{T}, 
        β       :: Vector{T}, 
        Σ       :: Matrix{T},
        L       :: Matrix{T},
        σ²      :: T,
        updater :: Bool = false
        ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    σ²inv   = inv(σ²)
    ####################
    # Evaluate objective
    ####################
    # form the q-by-q matrix: Lt Zt Z L
    copy!(obs.ltztzl, obs.ztz)
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.ltztzl) # O(q^3)
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.ltztzl) # O(q^3)        
    # form the q-by-q matrix: M = σ² I + Lt Zt Z L
    copy!(obs.storage_qq, obs.ltztzl)
    @inbounds for j in 1:q
        obs.storage_qq[j, j] += σ²
    end
    LAPACK.potrf!('U', obs.storage_qq) # O(q^3)
    # Zt * res
    updater && BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), copy!(obs.ztr, obs.zty)) # O(pq)
    # Lt * (Zt * res)
    BLAS.trmv!('L', 'T', 'N', L, copy!(obs.ltztr, obs.ztr))    # O(q^2)
    # storage_q = (Mchol.U') \ (Lt * (Zt * res))
    BLAS.trsv!('U', 'T', 'N', obs.storage_qq, copy!(obs.storage_q, obs.ltztr)) # O(q^3)
    # Xt * res = Xt * y - Xt * X * β
    updater && BLAS.gemv!('N', T(-1), obs.xtx, β, T(1), copy!(obs.xtr, obs.xty))
    # l2 norm of residual vector
    updater && (obs.rtr[1] = obs.yty - dot(obs.xty, β) - dot(obs.xtr, β))
    # assemble pieces
    logl::T = n * log(2π) + (n - q) * log(σ²) # constant term
    @inbounds for j in 1:q # log det term
        logl += 2log(obs.storage_qq[j, j])
    end
    qf    = abs2(norm(obs.storage_q)) # quadratic form term
    logl += (obs.rtr[1] - qf) * σ²inv 
    logl /= -2
    ######################################
    # TODO: Evaluate posterior mean and variance
    ######################################   
    
    #νγ
    copy!(obs.νγ, L)
    LAPACK.trtri!('L', 'N', obs.νγ)
    BLAS.gemm!('T', 'N', T(1), obs.νγ, obs.νγ, T(0), obs.storage_qq)
    BLAS.axpy!(1/σ², obs.ztz, obs.storage_qq)
    
    copy!(obs.νγ, obs.storage_qq)
    LAPACK.potrf!('U',obs.νγ) 
    LAPACK.potri!('U', obs.νγ)
    LinearAlgebra.copytri!(obs.νγ, 'U')
    
    #μγ
    BLAS.gemm!('N','N', T(1), obs.νγ, obs.ztr, T(0), obs.μγ)
    obs.μγ ./= σ²

    # Return
    ###################        
    return logl
end


logl!

It is a good idea to test correctness and efficiency of the single datum objective/posterior mean/var evaluator here. It's the same test datum in HW2 and HW4.

In [4]:
Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X = [ones(n) randn(n, p - 1)]
Z = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I # compound symmetry 
L  = Matrix(cholesky(Symmetric(Σ)).L)
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form the LmmObs object
obs = LmmObs(y, X, Z);

### Correctness

In [11]:
@show logl = logl!(obs, β, Σ, L, σ², true)
@show obs.μγ
@show obs.νγ;

logl = logl!(obs, β, Σ, L, σ², true) = -3261.9177559187583
obs.μγ = [-1.2467648280833787, -0.02448852130995821, -0.7973392260495257]
obs.νγ = [0.0007494365950329128 1.9689775091534504e-6 1.636132992910724e-6; 1.9689775091534504e-6 0.0007323744614646163 -1.0428608787480572e-5; 1.636132992910724e-6 -1.0428608787480572e-5 0.0007326509694024718]


You will lose all 20 points if following statement throws `AssertionError`.

In [12]:
@assert abs(logl - (-3261.9177559187597)) < 1e-8
@assert norm(obs.μγ - [-1.2467648280832386, 
        -0.02448852130995758, -0.7973392260495442]) < 1e-8
@assert norm(obs.νγ - [0.000749436595166797 1.9689775630068264e-6 1.6361330590481234e-6; 
        1.96897748759551e-6 0.0007323744616953198 -1.0428608786771824e-5; 
        1.6361330146430107e-6 -1.0428608802351851e-5 0.0007326509694599695]) < 1e-8

### Efficiency

Benchmark for efficiency.

In [13]:
bm_obj = @benchmark logl!($obs, $β, $Σ, $L, $σ², true)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.340 μs (0.00% GC)
  median time:      1.400 μs (0.00% GC)
  mean time:        1.405 μs (0.00% GC)
  maximum time:     5.080 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     10

My median runt time is 1.7μs. You will get full credit if the median run time is within 10μs. The points you will get are

In [14]:
clamp(10 / (median(bm_obj).time / 1e3) * 10, 0, 10)

10.0

In [ ]:
# # check for type stability
# @code_warntype logl!(obs, β, Σ, L, σ²)

In [ ]:
# using Profile

# Profile.clear()
# @profile for i in 1:10000; logl!(obs, β, Σ, L, σ²); end
# Profile.print(format=:flat)

## Q4. LmmModel type

We modify the `LmmModel` type in HW4 to hold all data points, model parameters, and intermediate arrays.

In [16]:
# define a type that holds LMM model (data + parameters)
struct LmmModel{T <: AbstractFloat}
    # data
    data :: Vector{LmmObs{T}}
    # parameters
    β        :: Vector{T}
    Σ        :: Matrix{T}
    L        :: Matrix{T}
    σ²       :: Vector{T}    
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    xty      :: Vector{T}
    xtr      :: Vector{T}
    ztr2     :: Vector{T}
    xtxinv   :: Matrix{T}
    ztz2     :: Matrix{T}
    store_1  :: Vector{T}
    store_2  :: Vector{T}
end  

"""
    LmmModel(data::Vector{LmmObs})

Create an LMM model that contains data and parameters.
"""
function LmmModel(obsvec::Vector{LmmObs{T}}) where T <: AbstractFloat
    # dims
    p      = size(obsvec[1].X, 2)
    q      = size(obsvec[1].Z, 2)
    # parameters
    β      = Vector{T}(undef, p)
    Σ      = Matrix{T}(undef, q, q)
    L      = Matrix{T}(undef, q, q)
    σ²     = Vector{T}(undef, 1)    
    # intermediate arrays
    xty    = zeros(T, p)
    xtr    = similar(xty)
    ztr2   = Vector{T}(undef, abs2(q))
    xtxinv = zeros(T, p, p)
    #new storage
    store_1 = similar(xty)
    store_2 = Vector{T}(undef, q)
    
    # pre-calculate \sum_i Xi^T Xi and \sum_i Xi^T y_i
    @inbounds for i in eachindex(obsvec)
        obs = obsvec[i]
        BLAS.axpy!(T(1), obs.xtx, xtxinv)
        BLAS.axpy!(T(1), obs.xty, xty)
    end
    # invert X'X
    LAPACK.potrf!('U', xtxinv)
    LAPACK.potri!('U', xtxinv)
    LinearAlgebra.copytri!(xtxinv, 'U')
    ztz2   = Matrix{T}(undef, abs2(q), abs2(q))
    LmmModel(obsvec, β, Σ, L, σ², xty, xtr, ztr2, xtxinv, ztz2, store_1, store_2)
end

LmmModel

## Q5. Implement EM update

Let's write the key function `update_em!` that performs one iteration of EM update.

In [24]:
"""
    update_em!(m::LmmModel, updater::Bool = false)

Perform one iteration of EM update. It returns the log-likelihood calculated 
from input `m.β`, `m.Σ`, `m.L`, and `m.σ²`. These fields are then overwritten 
by the next EM iterate. The fields `m.data[i].xtr`, `m.data[i].ztr`, and 
`m.data[i].rtr` are updated according to the resultant `m.β`. If `updater==true`, 
the function first updates `m.data[i].xtr`, `m.data[i].ztr`, and 
`m.data[i].rtr` according to `m.β`. If `updater==false`, it assumes these fields 
are pre-computed.
"""
function update_em!(m::LmmModel{T}, updater::Bool = false) where T <: AbstractFloat
    logl = zero(T)
    fill!(m.store_1, 0)
    
    # TODO: update m.β
    @inbounds for i in 1:length(m.data)
        dat = m.data[i]
        logl += logl!(dat, m.β, m.Σ, m.L, m.σ²[1], updater)
        BLAS.gemv!('T', T(-1), dat.ztx, dat.μγ, T(1), m.store_1)
    end
    BLAS.axpy!(T(1), m.xty, m.store_1)
    BLAS.gemv!('N', T(1), m.xtxinv, m.store_1, T(0), m.β)
    
    # TODO: update m.data[i].ztr, m.data[i].xtr, m.data[i].rtr
    @inbounds for i in 1:length(m.data)
        dat = m.data[i]
        copy!(dat.ztr, dat.zty)
        copy!(dat.xtr, dat.xty)
        BLAS.gemv!('N', T(-1), dat.ztx, m.β, T(1), dat.ztr)
        BLAS.gemv!('N', T(-1), dat.xtx, m.β, T(1), dat.xtr)
        dat.rtr[1] = dat.yty - dot(dat.xty, m.β) - dot(dat.xtr, m.β)
    end
    
    # TODO: update m.σ²
    ncount = 0 
    @inbounds for i in 1:length(m.data)
        dat = m.data[i]
        m.σ²[1] += dat.rtr[1]
        m.σ²[1] += dot(dat.ztz, dat.νγ)
        ncount += size(dat.X, 1)
        BLAS.gemv!('N', T(1), dat.ztz, dat.μγ, T(0), m.store_2)
        BLAS.axpy!(T(-2), dat.ztr, m.store_2)
        m.σ²[1] += dot(m.store_2, dat.μγ)
    end
    m.σ²[1] /= ncount
    
    # update m.Σ 
    fill!(m.Σ, 0)
    @inbounds for i in 1:length(m.data)
        dat = m.data[i]
        BLAS.axpy!(T(1), dat.νγ, m.Σ)
        BLAS.gemm!('N', 'T', T(1), dat.μγ, dat.μγ, T(1), m.Σ)
    end
    m.Σ ./= length(m.data)
    
    #update m.L
    copy!(m.L, m.Σ)
    LAPACK.potrf!('L', m.L)
    
    # return log-likelihood at input parameter values
    logl
end

update_em!

## Q6. (30 pts) Test data

Let's generate a fake longitudinal data set (same as HW4) to test our algorithm.

In [25]:
Random.seed!(257)

# dimension
m      = 1000 # number of individuals
ns     = rand(1500:2000, m) # numbers of observations per individual
p      = 5 # number of fixed effects, including intercept
q      = 3 # number of random effects, including intercept
obsvec = Vector{LmmObs{Float64}}(undef, m)
# true parameter values
βtrue  = [0.1; 6.5; -3.5; 1.0; 5]
σ²true = 1.5
σtrue  = sqrt(σ²true)
Σtrue  = Matrix(Diagonal([2.0; 1.2; 1.0]))
Ltrue  = Matrix(cholesky(Symmetric(Σtrue)).L)
# generate data
for i in 1:m
    # first column intercept, remaining entries iid std normal
    X = Matrix{Float64}(undef, ns[i], p)
    X[:, 1] .= 1
    @views Distributions.rand!(Normal(), X[:, 2:p])
    # first column intercept, remaining entries iid std normal
    Z = Matrix{Float64}(undef, ns[i], q)
    Z[:, 1] .= 1
    @views Distributions.rand!(Normal(), Z[:, 2:q])
    # generate y
    y = X * βtrue .+ Z * (Ltrue * randn(q)) .+ σtrue * randn(ns[i])
    # form a LmmObs instance
    obsvec[i] = LmmObs(y, X, Z)
end
# form a LmmModel instance
lmm = LmmModel(obsvec);

### Correctness

Evaluate log-likelihood and gradient at the true parameter values.

In [26]:
copy!(lmm.β, βtrue)
copy!(lmm.Σ, Σtrue)
copy!(lmm.L, Ltrue)
lmm.σ²[1] = σ²true
@show obj1 = update_em!(lmm, true)
@show lmm.β
@show lmm.Σ
@show lmm.L
@show lmm.σ²
println()
@show obj2 = update_em!(lmm, false)
@show lmm.β
@show lmm.Σ
@show lmm.L
@show lmm.σ²

obj1 = update_em!(lmm, true) = -2.8342752296337797e6
lmm.β = [0.10001001012634451, 6.498340323625657, -3.500511832401159, 1.0018187869990531, 5.001950303046149]
lmm.Σ = [2.0690699112526745 0.052607487006816334 0.03312753696311742; 0.052607487006816334 1.1219141710884746 0.05589997696166215; 0.03312753696311742 0.05589997696166215 1.0120388707897787]
lmm.L = [1.4384261924939612 0.052607487006816334 0.03312753696311742; 0.03657294846362942 1.0585729027937336 0.05589997696166215; 0.023030404435058625 0.05201123798073488 1.0043920063327105]
lmm.σ² = [1.502348868068465]

obj2 = update_em!(lmm, false) = -2.834264639755199e6
lmm.β = [0.100019726591263, 6.49833740466359, -3.5005129674774547, 1.0018219298211528, 5.001953628901834]
lmm.Σ = [2.0691178664656715 0.05256690007679155 0.03311271068552869; 0.05256690007679155 1.121787666414385 0.05589102985444454; 0.03311271068552869 0.05589102985444454 1.0120392092049062]
lmm.L = [1.438442861731279 0.05256690007679155 0.03311271068552869; 0.0365443087

1-element Vector{Float64}:
 1.5023529398031097

Test correctness. You will loss all 30 points if following code throws `AssertError`.

In [29]:
@assert abs(obj1 - (-2.8342752296337797e6)) < 1e-6
@assert abs(obj2 - (-2.8342646397564034e6)) < 1e-5

#My second object is not as precisely similar - but it works for 1e-5

### Efficiency

Test efficiency of EM update.

In [30]:
bm_emupdate = @benchmark update_em!($lmm, true) setup=(
    copy!(lmm.β, βtrue);
    copy!(lmm.Σ, Σtrue);
    copy!(lmm.L, Ltrue);
    lmm.σ²[1] = σ²true)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.796 ms (0.00% GC)
  median time:      1.887 ms (0.00% GC)
  mean time:        2.029 ms (0.00% GC)
  maximum time:     9.764 ms (0.00% GC)
  --------------
  samples:          2452
  evals/sample:     1

My median run time is 2.17ms. You will get full credit if your median run time is within 10ms. The points you will get are

In [31]:
clamp(10 / (median(bm_emupdate).time / 1e6) * 10, 0, 10)

10.0

### Memory

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [32]:
clamp(10 - median(bm_emupdate).memory / 100, 0, 10)

10.0

## Q7. Starting point

We use the same least squares estimates as in HW4 as starting point. 

In [33]:
"""
    init_ls!(m::LmmModel)

Initialize parameters of a `LmmModel` object from the least squares estimate. 
`m.β`, `m.L`, and `m.σ²` are overwritten with the least squares estimates.
"""
function init_ls!(m::LmmModel{T}) where T <: AbstractFloat
    p, q = size(m.data[1].X, 2), size(m.data[1].Z, 2)
    # LS estimate for β
    mul!(m.β, m.xtxinv, m.xty)
    # LS etimate for σ2 and Σ
    rss, ntotal = zero(T), 0
    fill!(m.ztz2, 0)
    fill!(m.ztr2, 0)    
    @inbounds for i in eachindex(m.data)
        obs = m.data[i]
        ntotal += length(obs.y)
        # update Xt * res
        BLAS.gemv!('N', T(-1), obs.xtx, m.β, T(1), copy!(obs.xtr, obs.xty))
        # rss of i-th individual
        rss += obs.yty - dot(obs.xty, m.β) - dot(obs.xtr, m.β)
        # update Zi' * res
        BLAS.gemv!('N', T(-1), obs.ztx, m.β, T(1), copy!(obs.ztr, obs.zty))
        # Zi'Zi ⊗ Zi'Zi
        kron_axpy!(obs.ztz, obs.ztz, m.ztz2)
        # Zi'res ⊗ Zi'res
        kron_axpy!(obs.ztr, obs.ztr, m.ztr2)
    end
    m.σ²[1] = rss / ntotal
    # LS estimate for Σ = LLt
    LAPACK.potrf!('U', m.ztz2)
    BLAS.trsv!('U', 'T', 'N', m.ztz2, m.ztr2)
    BLAS.trsv!('U', 'N', 'N', m.ztz2, m.ztr2)
    copyto!(m.Σ, m.ztr2)
    copy!(m.L, m.Σ)
    LAPACK.potrf!('L', m.L)
    for j in 2:q, i in 1:j-1
        m.L[i, j] = 0
    end
    m
end

"""
    kron_axpy!(A, X, Y)

Overwrite `Y` with `A ⊗ X + Y`. Same as `Y += kron(A, X)` but
more memory efficient.
"""
function kron_axpy!(
        A::AbstractVecOrMat{T},
        X::AbstractVecOrMat{T},
        Y::AbstractVecOrMat{T}
        ) where T <: Real
    m, n = size(A, 1), size(A, 2)
    p, q = size(X, 1), size(X, 2)
    @assert size(Y, 1) == m * p
    @assert size(Y, 2) == n * q
    @inbounds for j in 1:n
        coffset = (j - 1) * q
        for i in 1:m
            a = A[i, j]
            roffset = (i - 1) * p            
            for l in 1:q
                r = roffset + 1
                c = coffset + l
                for k in 1:p                
                    Y[r, c] += a * X[k, l]
                    r += 1
                end
            end
        end
    end
    Y
end

kron_axpy!

In [34]:
init_ls!(lmm)
@show lmm.β
@show lmm.Σ
@show lmm.L
@show lmm.σ²

lmm.β = [0.12814248060847558, 6.497945035096696, -3.502138774770135, 1.0026609224782135, 5.002697971721905]
lmm.Σ = [2.0796224159409324 0.05523492575567489 0.04021508338255691; 0.055234925755674895 1.1137314752104626 0.06543803114908457; 0.04021508338255691 0.06543803114908459 1.0074558313277848]
lmm.L = [1.4420896005245072 0.0 0.0; 0.038302006848662674 1.0546394793870688 0.0; 0.027886674564416902 0.06103499518749489 1.0014753586943543]
lmm.σ² = [5.701709885959428]


1-element Vector{Float64}:
 5.701709885959428

## Q8. Estimation by EM

We write a function `fit!` that implements the EM algorithm for estimating LMM.

In [35]:
"""
    fit!(m::LmmModel)

Fit an `LmmModel` object by MLE using a EM algorithm. Start point 
should be provided in `m.β`, `m.σ²`, `m.L`.
"""
function fit!(
        m       :: LmmModel;
        maxiter :: Integer       = 10_000,
        ftolrel :: AbstractFloat = 1e-12,
        prtfreq :: Integer       = 0
    )
    obj = update_em!(m, true)
    for iter in 0:maxiter
        obj_old = obj
        # EM update
        obj = update_em!(m, false)
        # print obj
        prtfreq > 0 && rem(iter, prtfreq) == 0 && println("iter=$iter, obj=$obj")
        # check monotonicity
        obj < obj_old && (@warn "monotoniciy violated")
        # check convergence criterion
        (obj - obj_old) < ftolrel * (abs(obj_old) + 1) && break
        # warning about non-convergence
        iter == maxiter && (@warn "maximum iterations reached")
    end
    m
end


fit!

## Q9. (20 pts) Test drive

Now we can run our EM algorithm to compute the MLE.

In [36]:
# initialize from least squares
init_ls!(lmm)

@time fit!(lmm, prtfreq = 100);

println("objective value at solution: ", update_em!(lmm)); println()
println("solution values:")
@show lmm.β
@show lmm.σ²
@show lmm.L * transpose(lmm.L)

iter=0, obj=-2.8342745400391878e6
iter=100, obj=-2.834264506414603e6
iter=200, obj=-2.8342645060447953e6
iter=300, obj=-2.834264505706769e6
iter=400, obj=-2.834264505397895e6
  0.830328 seconds (150.49 k allocations: 8.484 MiB, 3.82% compilation time)
objective value at solution: -2.8342645052620987e6

solution values:
lmm.β = [0.1273018496823157, 6.498337399856076, -3.5005129709263225, 1.001821934100821, 5.001953634612177]
lmm.σ² = [1.5023529470461496]
lmm.L * transpose(lmm.L) = [2.072456944712887 0.11107604898782195 0.06809821196589814; 0.11107604898782195 1.1249111478199008 0.11202811217243856; 0.06809821196589814 0.11202811217243856 1.0120393497526112]


3×3 Matrix{Float64}:
 2.07246    0.111076  0.0680982
 0.111076   1.12491   0.112028
 0.0680982  0.112028  1.01204

### Correctness

You get 10 points if the following code does not throw `AssertError`.

In [37]:
# objective at solution should be close enough to the optimal
@assert update_em!(lmm) > -2.83426451e6

### Efficiency

My median run time 1.8s. You get 10 points if your median run time is within 5s.

In [38]:
bm_em = @benchmark fit!($lmm) setup = (init_ls!(lmm))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     757.700 ms (0.00% GC)
  median time:      772.333 ms (0.00% GC)
  mean time:        772.028 ms (0.00% GC)
  maximum time:     792.891 ms (0.00% GC)
  --------------
  samples:          7
  evals/sample:     1

In [39]:
# this is the points you get
clamp(5 / (median(bm_em).time / 1e9) * 10, 0, 10)

10.0

## Q10. (10 pts) EM vs Newton type algorithms

Contrast EM algorithm to the Newton type algorithms (gradient free, gradient based, using Hessian) in HW4, in terms of the stability, convergence rate (how fast the algorithm is converging),  final objective value, total run time, derivation, and implementation efforts. 

EM algorithm is much slower than Newton type algorithms by a factor of 4-5 on my machine. However, it appears that Newton-type algorithms converge faster and were closer to "correctness" than my EM example (had to broaden error range to 1e-5 when checking EM correctness). Both derivation and implementation-wise, EM is more difficult to write code for. This is probably because Newton's algorithm has many concepts I've seen before and were more intuitive to me from my previous math background.